In [4]:
!wget https://gitlab.fing.edu.uy/gsi/web-application-attacks-datasets/raw/master/ecml_pkdd/learning_dataset.xml

--2019-11-01 10:29:49--  https://gitlab.fing.edu.uy/gsi/web-application-attacks-datasets/raw/master/ecml_pkdd/learning_dataset.xml
Résolution de gitlab.fing.edu.uy (gitlab.fing.edu.uy)… 164.73.47.230
Connexion à gitlab.fing.edu.uy (gitlab.fing.edu.uy)|164.73.47.230|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 106610653 (102M) [text/plain]
Sauvegarde en : « learning_dataset.xml »

learning_dataset.xm 100%[===================>] 101,67M  1,10MB/s    ds 2m 19s  

2019-11-01 10:32:11 (747 KB/s) — « learning_dataset.xml » sauvegardé [106610653/106610653]



In [5]:
import xml.etree.ElementTree as et

xtree = et.parse("learning_dataset.xml")
xroot = xtree.getroot()

In [7]:
import re
import numpy as np
from numpy import array
import csv
import random
import pandas as pd



docs=[]
labels={'Valid':0, 'XSS':1, 'SqlInjection':2, 'LdapInjection':3,
        'XPathInjection':4, 'PathTransversal':5, 'OsCommanding':6, 'SSI':7}
max_length=0
for node in xroot: 
    sample_id = node.attrib.get("id")

    request = node.find('{http://www.example.org/ECMLPKDD}request')
    clazz = node.find('{http://www.example.org/ECMLPKDD}class')
    type_ = clazz.find('{http://www.example.org/ECMLPKDD}type')
    method = request.find('{http://www.example.org/ECMLPKDD}method')
    protocol = request.find('{http://www.example.org/ECMLPKDD}protocol')
    uri = request.find('{http://www.example.org/ECMLPKDD}uri')
    query = request.find('{http://www.example.org/ECMLPKDD}query')
    headers = request.find('{http://www.example.org/ECMLPKDD}headers')
    
    if query is not None: 
        query_text = "?"+query.text
    else:
        query_text = ""
    headers = request.find('{http://www.example.org/ECMLPKDD}headers')
    body = request.find('{http://www.example.org/ECMLPKDD}body')
    if body is not None:
        body_text= body.text
    else:
        body_text = ""
    
    
    request_text = method.text+" "+uri.text+query_text+" "+protocol.text+"\n"+headers.text+"\n"
    if len(body_text)>1:
        request_text = request_text+ "\n"+body_text+"\n"
     
    label = labels[type_.text]
    docs.append( (request_text,str(label)) )
    
    if len(request_text)>max_length:
            max_length=len(request_text)

df  = pd.DataFrame.from_records(docs,columns=['request','label'])
df['label'].apply(str)

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])



#print(len(df.loc[df['label'] != '0']))

print("Training data length: "+str(len(train['request'])))
print("Validate data length: "+str(len(validate['request'])))
print("Test data length: "+str(len(test['request'])))

train.to_csv('../_train.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
validate.to_csv('../_validate.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
test.to_csv('../_test.csv',header=False,index=False,quoting=csv.QUOTE_ALL,columns=['label','request'])
print("MAX input length: "+str(max_length))
    
#print number of samples per class  in training data
train['label'].value_counts()  


Training data length: 30069
Validate data length: 10023
Test data length: 10024
MAX input length: 2423


0    20988
2     1401
3     1396
6     1374
5     1372
4     1366
1     1091
7     1081
Name: label, dtype: int64